In [ ]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import pickle
from dateutil.parser import *
import time
import logging
import os
import pprint
import pytz
import sys
import random

In [ ]:
#suwon2912
API_KEY = 'x'
ACCOUNT_ID = 'x'

OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [ ]:
def pushbullet_message(title, body = None):
    msg = {"type": "note", "title": title, "body": body}
    TOKEN = 'x'
    resp = requests.post('https://api.pushbullet.com/v2/pushes', 
                         data=json.dumps(msg),
                         headers={'Authorization': 'Bearer ' + TOKEN,
                                  'Content-Type': 'application/json'})
    #if resp.status_code != 200:
        #raise Exception('Error',resp.status_code)
    #else:
        #print ('Message sent')

def slack_msg(text):
    url = "x"
    message = (text)
    title = (f"New Incoming Message :zap:")
    slack_data = {
        "username": "NotificationBot",
        "icon_emoji": ":satellite:",
        "channel" : "oanda_notify",
        "attachments": [
            {
                "color": "#9733EE",
                "fields": [
                    {
                        "title": title,
                        "value": message,
                        "short": "false",
                    }
                ]
            }
        ]
    }
    byte_length = str(sys.getsizeof(slack_data))
    headers = {'Content-Type': "application/json", 'Content-Length': byte_length}
    response = requests.post(url, data=json.dumps(slack_data), headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)

In [ ]:
def time_utc():
    return dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc)

def get_utc_dt_from_string(date_str):
    d = parse(date_str)
    return d.replace(tzinfo=dt.timezone.utc)

def astimezone(row):
    return row.astimezone(pytz.timezone('Singapore'))

#getting the pkl file of certain pair
def get_his_data_filename(pair, granularity):
    return f"{pair}_{granularity}_break.pkl"

In [ ]:
class OandaTrade():
    def __init__(self,oanda_ob):
        
        self.unrealizedPL = float(oanda_ob['unrealizedPL'])
        self.currentUnits = int(oanda_ob['currentUnits'])
        self.trade_id= int(oanda_ob['id'])
        self.openTime = parse(oanda_ob['openTime'])
        self.instrument = oanda_ob['instrument']
        
    def __repr__(self):
        return str(vars(self))

    @classmethod
    def TradeFromAPI(cls, api_object):
        return OandaTrade(api_object)

In [ ]:
class OandaAPI():

    def __init__(self):
        self.session = requests.Session()    
    
    #connection link to server
    def make_request(self, url, params={}, added_headers=None, verb='get', data=None, code_ok=200):

        headers = SECURE_HEADER

        if added_headers is not None:   
            for k in added_headers.keys():
                headers[k] = added_headers[k]
                
        try:
            response = None
            if verb == 'post':
                response = self.session.post(url,params=params,headers=headers,data=data)
            elif verb == 'put':
                response = self.session.put(url,params=params,headers=headers,data=data)
            else:
                response = self.session.get(url,params=params,headers=headers,data=data)

            status_code = response.status_code

            if status_code == code_ok:
                json_response = response.json()
                return status_code, json_response
            
            else:
                return status_code, None  
                
        except:
            print('make_request error 1')
            pushbullet_message('make_request error 1')
            slack_msg('make_request error 1')
            time.sleep(70)
            try:
                response = None
                if verb == 'post':
                    response = self.session.post(url,params=params,headers=headers,data=data)
                elif verb == 'put':
                    response = self.session.put(url,params=params,headers=headers,data=data)
                else:
                    response = self.session.get(url,params=params,headers=headers,data=data)
                status_code = response.status_code
                
                if status_code == code_ok:
                    json_response = response.json()
                    return status_code, json_response
            
                else:
                    return status_code, None  
            except:
                print("make_request error 2")
                pushbullet_message('make_request error 2')
                slack_msg('make_request error 2')
                return 400, None
                
    #get instruments list
    def fetch_instruments(self):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"
        status_code, data = self.make_request(url)
        return status_code, data
    
    #change instruments list to df
    def get_instruments_df(self):
        status_code, data = self.fetch_instruments()
        if status_code == 200:
            df = pd.DataFrame.from_dict(data['instruments'])
            return df[['name', 'type', 'displayName', 'pipLocation', 'marginRate']]
        else:
            return None
    
    #get candles from server and change to df with candles_to_df
    def fetch_candles(self, pair_name, count=None, granularity="H1", date_from=None, date_to=None, as_df=True):
        url = f"{OANDA_URL}/instruments/{pair_name}/candles"

        params = dict(
            granularity = granularity,
            price = "MBA"
        )
        
        if date_from is not None and date_to is not None:
            params['to'] = int(date_to.timestamp())
            params['from'] = int(date_from.timestamp())
        elif count is not None:
            params['count'] = count
        else:
            params['count'] = 300
            
        try:
            status_code, data = self.make_request(url, params=params)
        
        except:
            
            try:
                print('GET error 1, sleep')
                pushbullet_message('GET error 1')
                slack_msg('GET error 1')
                time.sleep(70)
                status_code, data = self.make_request(url, params=params)
                
            
            except:
                print('GET error 2, sleep')
                pushbullet_message('GET error 2')
                slack_msg('GET error 2')
                time.sleep(70)
                status_code, data = self.make_request(url, params=params)

        if status_code != 200:
            print(status_code)
            pushbullet_message('GET error 3',status_code)
            slack_msg(f'GET error 3, status code: {status_code}')
            return status_code, None

        return status_code, OandaAPI.candles_to_df(data['candles'])

    def place_trade(self,pair, units, price, take_profit, stop_loss, price_bound):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/orders"

        data = {
            "order": {
                "units": units,
                "instrument": pair,
                "price": str(price),
                "timeInForce": "GTD",
                "type": "STOP",
                "gtdTime": str(time_utc()+dt.timedelta(46)),
                "positionFill": "DEFAULT",
                "triggerCondition": "DEFAULT",
                "takeProfitOnFill": {"timeInForce": "GTC", "price": str(take_profit)},
                "stopLossOnFill": {"timeInForce": "GTC", "price": str(stop_loss)},
                "priceBound": str(price_bound)

            }
        }
        
        #make a trade with the data above
        status_code, json_data = self.make_request(url,verb = 'post',data = json.dumps(data), code_ok = 201)
        
        if status_code!=201:
            data = {
            "order": {
                "units": units,
                "instrument": pair,
                "price": str(round(price,4)),
                "timeInForce": "GTD",
                "type": "STOP",
                "gtdTime": str(time_utc()+dt.timedelta(46)),
                "positionFill": "DEFAULT",
                "triggerCondition": "DEFAULT",
                "takeProfitOnFill": {"timeInForce": "GTC", "price": str(round(take_profit,4))},
                "stopLossOnFill": {"timeInForce": "GTC", "price": str(round(stop_loss,4))},
                "priceBound": str(round(price_bound,4))

            }
        }
            status_code, json_data = self.make_request(url, verb='post', data=json.dumps(data), code_ok=201)

            if status_code != 201:
                data = {
                    "order": {
                        "units": units,
                        "instrument": pair,
                        "price": str(round(price,3)),
                        "timeInForce": "GTD",
                        "type": "STOP",
                        "gtdTime": str(time_utc()+dt.timedelta(46)),
                        "positionFill": "DEFAULT",
                        "triggerCondition": "DEFAULT",
                        "takeProfitOnFill": {"timeInForce": "GTC", "price": str(round(take_profit,3))},
                        "stopLossOnFill": {"timeInForce": "GTC", "price": str(round(stop_loss,3))},
                        "priceBound": str(round(price_bound,3))
                    }
                }
                status_code, json_data = self.make_request(url, verb='post', data=json.dumps(data), code_ok=201)

                if status_code != 201:
                    print('error making trade',status_code, pair, dt.datetime.now())
                    pushbullet_message('error making trade',f'{pair},{status_code}')
                    slack_msg(f'error making trade, status code: {status_code}, pair: {pair}, time: {dt.datetime.now()}')

        trade_id = None
        if 'orderCreateTransaction' in json_data:
            trade_id =  int(json_data['orderCreateTransaction']['id'])
            price_str =  json_data['orderCreateTransaction']['price']
            units = int(json_data['orderCreateTransaction']['units'])
            print('trade done')
        
        print (trade_id, status_code, price_str)
                
    #run request to close trade
    def close_trade(self,trade_id):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/trades/{trade_id}/close"
        status_code, json_data = self.make_request(url, verb='put', code_ok=200)
        if status_code !=200:
            print(status_code)
            return False
        print ('close trade')
        return True
    
    #returns all open trades as OandaTrade object
    def open_trades(self):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/openTrades"
        status_code, data = self.make_request(url)
        
        if status_code !=200:
            return [], False
        #could be that the code work but we dont have any open trades. will still return false. so return true as long as 
        #'trades' not in data
        if 'trades' not in data:
            return [], True
        
        trades = [OandaTrade.TradeFromAPI(x) for x in data['trades']]
        
        return trades,True
    
    #change candles from fetch candles to a dataframe
    @classmethod
    def candles_to_df(cls, json_data):
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']

        our_data = []
        for candle in json_data:
            if candle['complete'] == False:
                continue
            new_dict = {}
            new_dict['time'] = candle['time']
            new_dict['volume'] = candle['volume']
            for price in prices:
                for oh in ohlc:
                    new_dict[f"{price}_{oh}"] = float(candle[price][oh])
            our_data.append(new_dict)
        df = pd.DataFrame.from_dict(our_data)
        
        return df
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

if __name__ == "__main__":
    api = OandaAPI()
    #res, df = api.fetch_candles("EUR_USD",date_to = time_utc(), date_from = time_utc()- dt.timedelta(35))
    #display(df.head())
    #print(api.last_complete_candle('EUR_USD'))
    trades, ok = api.open_trades()
    if ok == True:
         [print(t) for t in trades]
         print(ok)

#api=OandaAPI()
#api.set_sl_tp(price = 1.5,'TAKE_PROFIT','1088')
#api = OandaAPI()
#api.place_trade('GBP_JPY',units = 50,price = 155.6337,take_profit = 155.6637, stop_loss= 155.3350)

In [ ]:
data_library = {
    'GBP_USD' :  { "pair" :  'GBP_USD', "units": 6000, "pip": 0.0001, "pips_tp": 55, "pips_sl": 20, "pips_exceed": 5,"pips_bound": 10},
    'USD_JPY' :  { "pair" :  'USD_JPY', "units": 6000, "pip": 0.01, "pips_tp": 55, "pips_sl": 20, "pips_exceed": 5,"pips_bound": 10},
    'AUD_JPY' :  { "pair" :  'AUD_JPY', "units": 3000, "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'USD_CAD' :  { "pair" :  'USD_CAD', "units": 3000, "pip": 0.0001, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'GBP_JPY' :  { "pair" :  'GBP_JPY', "units": 3000, "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'XAU_USD' :  { "pair" :  'XAU_USD', "units": 3,    "pip": 0.1, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'BCO_USD' :  { "pair" :  'BCO_USD', "units": 30,    "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15},
    'WTICO_USD' :  { "pair" :  'WTICO_USD', "units": 30,    "pip": 0.01, "pips_tp": 110, "pips_sl": 40, "pips_exceed": 10,"pips_bound": 15}

    }

#'XAU_USD' :  { "pair" :  'XAU_USD', "units": 1,    "pip": 0.1}

In [ ]:
def get_res_sup(temp_df,candles,pair):
    api= OandaAPI()

    temp_df['time']= [parse(x) for x in temp_df['time']]

    log_df = pd.read_pickle('break_stop_log.pkl')

    value_range = [0]*candles
    value_range2 = [100000]*candles
    date_range = [0]*candles
    max_counter =0
    min_counter = 0
    
    for i in temp_df.index:
        max_value = max(value_range)
        min_value = min(value_range2)
    
        value_range = value_range[1:candles]
        value_range.append(round(temp_df['mid_c'][i],4))
    
        value_range2 = value_range2[1:candles]
        value_range2.append(round(temp_df['mid_c'][i],4))
    
        date_range=date_range[1:candles]
        date_range.append(temp_df['time'][i])
     
        if max_value==max(value_range):
            max_counter+=1
        
        if max_value<max(value_range):
            max_counter =0
     
        if min_value==min(value_range2):
            min_counter+=1
         
        if min_value>min(value_range2):
            min_counter =0
         
        if max_counter ==(candles/2):
            date_index = value_range.index(max_value)
            date_pivot = astimezone(date_range[date_index])

            if ((log_df['PAIR']==pair) & (log_df['sure_level']==max_value) & (log_df['time']== date_pivot)).any() ==False:
                
                if date_pivot>temp_df['time'][0]+dt.timedelta(5):
                    api.place_trade(pair, 
                                data_library[pair]['units'], 
                                max_value+data_library[pair]['pip']*data_library[pair]['pips_exceed'],
                                take_profit = max_value+data_library[pair]['pip']*data_library[pair]['pips_tp'],
                                stop_loss = max_value-data_library[pair]['pip']*data_library[pair]['pips_sl'],
                                price_bound = max_value+data_library[pair]['pip']*data_library[pair]['pips_bound'])
                    temp_df2 = pd.DataFrame({'time':[date_pivot],
                                        'date_added':[dt.datetime.now()],
                                        'PAIR':[pair],
                                        'sure_level':[max_value],
                                        'decision': [1]})
                    #temp_df2['time'] = temp_df2['time'].apply(astimezone)
                    log_df = log_df.append(temp_df2)
                    log_df.to_pickle('break_stop_log.pkl')

                    entry_price = max_value+data_library[pair]['pip']*data_library[pair]['pips_exceed']
                    pushbullet_message('trade done',f'{pair} buy stop {round(entry_price,5)}, {(dt.datetime.now()+dt.timedelta(46)).day}-{(dt.datetime.now()+dt.timedelta(46)).month}')
                    slack_msg(f'trade done, pair: {pair}, buy stop: {round(entry_price,5)}, {(dt.datetime.now()+dt.timedelta(46)).day}-{(dt.datetime.now()+dt.timedelta(46)).month}')
                    print(pair, 'buy_stop_price: ', entry_price)
                
        if min_counter ==(candles/2):
            date_index = value_range2.index(min_value)
            date_pivot = astimezone(date_range[date_index])

            if ((log_df['PAIR']==pair) & (log_df['sure_level']==min_value) & (log_df['time']== date_pivot)).any() ==False:

                if date_pivot>temp_df['time'][0]+dt.timedelta(5):
                    api.place_trade(pair, 
                                -data_library[pair]['units'], 
                                min_value-data_library[pair]['pip']*data_library[pair]['pips_exceed'],
                                take_profit = min_value-data_library[pair]['pip']*data_library[pair]['pips_tp'],
                                stop_loss = min_value+data_library[pair]['pip']*data_library[pair]['pips_sl'],
                                price_bound = min_value-data_library[pair]['pip']*data_library[pair]['pips_bound'])
                    temp_df2 = pd.DataFrame({'time':[date_pivot],
                                        'date_added':[dt.datetime.now()],
                                        'PAIR':[pair],
                                        'sure_level':[min_value],
                                        'decision': [-1]})
                    #temp_df2['time'] = temp_df2['time'].apply(astimezone)
                    log_df = log_df.append(temp_df2)
                    log_df.to_pickle('break_stop_log.pkl')

                    entry_price = min_value-data_library[pair]['pip']*data_library[pair]['pips_exceed']
                    pushbullet_message('trade done', f'{pair} sell stop {round(entry_price,5)}, {(dt.datetime.now()+dt.timedelta(46)).day}-{(dt.datetime.now()+dt.timedelta(46)).month}')
                    slack_msg(f'trade done, pair: {pair}, sell stop: {round(entry_price,5)}, {(dt.datetime.now()+dt.timedelta(46)).day}-{(dt.datetime.now()+dt.timedelta(46)).month}')
                    print(pair, 'sell_stop_price: ', entry_price)

In [ ]:
def create_file(pair, granularity, api):
    candle_count = 2000
    time_step = INCREMENTS[granularity] * candle_count

    end_date = time_utc()
    date_from = time_utc()- dt.timedelta(46)
    #"2020-06-30 23:59:59"
    candle_dfs = []

    date_to = date_from
    #change date_to to whatever date_from is plus the time delta. after first loop is done, our date_to >
    # will be updated again and it takes the next chunk
    while date_to < end_date:
        date_to = date_from + dt.timedelta(minutes=time_step)
        if date_to > end_date:
            date_to = end_date
                
        code, df = api.fetch_candles(pair,
                granularity=granularity,
                date_from=date_from,
                date_to=date_to,as_df=True)
        if df is not None and df.empty ==False:
            candle_dfs.append(df)

        elif code != 200:
            print("ERROR", pair, granularity, date_from, date_to)
            pushbullet_message('create_file error 1', f'{code}, {pair}, {date_from}, {date_to}')
            slack_msg(f'create_file error 1, code: {code}, pair: {pair}, date from: {date_from}, date to: {date_to}')
            
            break
        date_from = date_to
    #put all the chunks of data into the list into a df
    final_df = pd.concat(candle_dfs)        
    final_df.drop_duplicates(subset='time', inplace=True) 
    final_df.sort_values(by='time', inplace=True)
    final_df.to_pickle(f'downloaded data/{get_his_data_filename(pair, granularity)}')
    print(f"{pair} {granularity} {final_df.iloc[0].time}  {final_df.iloc[-1].time} {dt.datetime.now().hour}:{dt.datetime.now().minute}")

In [ ]:
def process_time():
    d = dt.datetime.utcnow().time()
    d1= dt.time(20,55)
    d2 = dt.time(21,0)
    api = OandaAPI()

    if d>d1 and d<d2:
        trades, ok = api.open_trades()
        if ok == True:
            ok1 = [api.close_trade(t.trade_id) for t in trades]
            if ok1 == False:
                print ('fail to close at end day')

In [ ]:
SLEEP = 10.0
INCREMENTS = {
    'M1':1
    #'M5' : 5,
    #'H1' : 60,
    #H4' : 240
}

#for each pair, create a file
def run_collection():
 while True:
    print ('running...')
    pair_list = ['GBP_USD','USD_JPY','AUD_JPY','USD_CAD','GBP_JPY','XAU_USD','BCO_USD','WTICO_USD']
    #'GBP_USD','USD_JPY','AUD_JPY','USD_CAD','GBP_JPY','XAU_USD'
    #['GBP_JPY','GBP_USD','EUR_JPY','EUR_USD','USD_CAD','XAU_USD','AUD_JPY','AUD_CAD','EUR_AUD','GBP_AUD','CAD_JPY','USD_JPY']
    api = OandaAPI()
    for g in INCREMENTS.keys():
        for pair in pair_list:
            create_file(pair, g, api)
            temp_df = pd.read_pickle(f'downloaded data/{get_his_data_filename(pair,g)}')
            temp_df.reset_index(drop=True,inplace=True)

            get_res_sup(temp_df,2880,pair)
            #process_time()
    time.sleep(SLEEP)
            
if __name__ == "__main__":
    run_collection()